In [1]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import Markdown, display
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    Expression,
    LogVectorOutput,
    MakeVectorVariable,
    MeshcatVisualizer,
    MultibodyPlant,
    Parser,
    RigidTransform_,
    Simulator,
    SpatialInertia_,
    StartMeshcat,
    ToLatex,
    UnitInertia_,
    Variable,
    VectorSystem,
)

from underactuated import (
    ConfigureParser,
    ManipulatorDynamics,
    running_as_notebook,
)

if running_as_notebook:
    mpld3.enable_notebook()

In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [5]:
def double_pendulum_demo():
    # Set up a block diagram with the robot (dynamics) and a visualization block.
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

    # Load the double pendulum from Universal Robot Description Format
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl(
        "package://underactuated/models/double_pendulum.urdf"
    )
    plant.Finalize()

    builder.ExportInput(plant.get_actuation_input_port())

    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    meshcat.Set2dRenderMode(xmin=-2.8, xmax=2.8, ymin=-2.8, ymax=2.8)

    logger = LogVectorOutput(plant.get_state_output_port(), builder)

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)

    if running_as_notebook:
        simulator.set_target_realtime_rate(1.0) # running it in real time

    # Set the initial conditions
    context = simulator.get_mutable_context()
    context.SetContinuousState(
        [1.5, -1.5, 0.0, 0.0]
    )  # (theta1, theta2, theta1dot, theta2dot)
    diagram.get_input_port(0).FixValue(
        context, [0.0, 0.0]
    )  # Zero input torques

    # Simulate
    simulator.AdvanceTo(10.0)

    # Plot the results
    plt.figure()
    fields = ["shoulder", "elbow"]
    log = logger.FindLog(context)
    for i in range(2):
        plt.subplot(2, 1, i + 1)
        plt.plot(log.sample_times(), log.data()[(i, i + 2), :].transpose())
        plt.legend(["position", "velocity"])
        plt.xlabel("t")
        plt.ylabel(fields[i])
        plt.grid(True)
    display(mpld3.display())


double_pendulum_demo()

In [6]:

display(Markdown("$M = " + ToLatex(M, precision=2) + "$"))
display(Markdown("$Cv = " + ToLatex(Cv, precision=2) + "$"))
display(Markdown("$\\tau_G = " + ToLatex(tauG, precision=2) + "$"))
display(Markdown("$B = " + ToLatex(B, precision=2) + "$"))
display(Markdown("$\\tau_{ext} = " + ToLatex(tauExt, precision=2) + "$"))

$M = \begin{bmatrix} 5 & 2 \\ 2 & 1 \end{bmatrix}$

$Cv = \begin{bmatrix} 0 \\ 0 \end{bmatrix}$

$\tau_G = \begin{bmatrix} -0 \\ -0 \end{bmatrix}$

$B = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}$

$\tau_{ext} = \begin{bmatrix} 0 \\ 0 \end{bmatrix}$

In [5]:
# Evaluate the dynamics symbolically
def SymbolicManipulatorEquations():
    plant = MultibodyPlant(time_step=0)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl(
        "package://underactuated/models/double_pendulum.urdf"
    )
    plant.Finalize()

    sym_plant = plant.ToSymbolic()
    sym_context = sym_plant.CreateDefaultContext()

    # State variables
    q = MakeVectorVariable(2, "q")
    v = MakeVectorVariable(2, "v")
    vd = MakeVectorVariable(2, "\dot{v}")

    # Parameters
    m = MakeVectorVariable(2, "m")
    l = MakeVectorVariable(2, "l")

    # TODO: pending https://github.com/RobotLocomotion/drake/issues/17245
    # g = Variable("g")
    # sym_plant.mutable_gravity_field().SetGravityVector(sym_context, [0, 0, -g])

    upper_arm = sym_plant.GetBodyByName("upper_arm")
    inertia = SpatialInertia_[Expression](
        m[0],
        [0, 0, -l[0]],
        UnitInertia_[Expression](l[0] * l[0], l[0] * l[0], 0),
    )
    upper_arm.SetSpatialInertiaInBodyFrame(sym_context, inertia)
    lower_arm = sym_plant.GetBodyByName("lower_arm")
    inertia = SpatialInertia_[Expression](
        m[1],
        [0, 0, -l[1]],
        UnitInertia_[Expression](l[1] * l[1], l[1] * l[1], 0),
    )
    lower_arm.SetSpatialInertiaInBodyFrame(sym_context, inertia)
    elbow_frame = sym_plant.GetJointByName("elbow").frame_on_parent()
    elbow_frame.SetPoseInParentFrame(
        sym_context, RigidTransform_[Expression]([0, 0, -l[0]])
    )
    (M, Cv, tauG, B, tauExt) = ManipulatorDynamics(
        sym_plant, q, v, sym_context
    )
    display(Markdown("$M = " + ToLatex(M, precision=2) + "$"))
    display(Markdown("$Cv = " + ToLatex(Cv, precision=2) + "$"))
    display(Markdown("$\\tau_G = " + ToLatex(tauG, precision=2) + "$"))
    display(Markdown("$B = " + ToLatex(B, precision=2) + "$"))
    display(Markdown("$\\tau_{ext} = " + ToLatex(tauExt, precision=2) + "$"))


SymbolicManipulatorEquations()

$M = \begin{bmatrix} (m_{0} l_{0}^{2} + m_{1} l_{0} l_{1} ( - \sin{q_{0}} \sin{q_{1}} + \cos{q_{0}} \cos{q_{1}}) \cos{q_{0}} + m_{1} l_{0} l_{1} (\sin{q_{0}} \cos{q_{1}} + \sin{q_{1}} \cos{q_{0}}) \sin{q_{0}} + m_{1} l_{1}^{2} + l_{0} (m_{1} l_{0} \sin{q_{0}} + m_{1} l_{1} (\sin{q_{0}} \cos{q_{1}} + \sin{q_{1}} \cos{q_{0}})) \sin{q_{0}} - l_{0} ( - m_{1} l_{0} \cos{q_{0}} - m_{1} l_{1} ( - \sin{q_{0}} \sin{q_{1}} + \cos{q_{0}} \cos{q_{1}})) \cos{q_{0}}) & (m_{1} l_{0} l_{1} ( - \sin{q_{0}} \sin{q_{1}} + \cos{q_{0}} \cos{q_{1}}) \cos{q_{0}} + m_{1} l_{0} l_{1} (\sin{q_{0}} \cos{q_{1}} + \sin{q_{1}} \cos{q_{0}}) \sin{q_{0}} + m_{1} l_{1}^{2}) \\ (m_{1} l_{0} l_{1} ( - \sin{q_{0}} \sin{q_{1}} + \cos{q_{0}} \cos{q_{1}}) \cos{q_{0}} + m_{1} l_{0} l_{1} (\sin{q_{0}} \cos{q_{1}} + \sin{q_{1}} \cos{q_{0}}) \sin{q_{0}} + m_{1} l_{1}^{2}) & m_{1} l_{1}^{2} \end{bmatrix}$

$Cv = \begin{bmatrix} ( - v_{0}^{2} m_{1} l_{0} l_{1} ( - \sin{q_{0}} \sin{q_{1}} + \cos{q_{0}} \cos{q_{1}}) \sin{q_{0}} + v_{0}^{2} m_{1} l_{0} l_{1} (\sin{q_{0}} \cos{q_{1}} + \sin{q_{1}} \cos{q_{0}}) \cos{q_{0}} - l_{0} (v_{0}^{2} m_{1} l_{0} \sin{q_{0}} + m_{1} l_{1} (v_{0} + v_{1})^{2} (\sin{q_{0}} \cos{q_{1}} + \sin{q_{1}} \cos{q_{0}})) \cos{q_{0}} + l_{0} (v_{0}^{2} m_{1} l_{0} \cos{q_{0}} + m_{1} l_{1} (v_{0} + v_{1})^{2} ( - \sin{q_{0}} \sin{q_{1}} + \cos{q_{0}} \cos{q_{1}})) \sin{q_{0}}) \\ ( - v_{0}^{2} m_{1} l_{0} l_{1} ( - \sin{q_{0}} \sin{q_{1}} + \cos{q_{0}} \cos{q_{1}}) \sin{q_{0}} + v_{0}^{2} m_{1} l_{0} l_{1} (\sin{q_{0}} \cos{q_{1}} + \sin{q_{1}} \cos{q_{0}}) \cos{q_{0}}) \end{bmatrix}$

$\tau_G = \begin{bmatrix} ( - 9.81m_{0} l_{0} \sin{q_{0}} - 9.81m_{1} l_{0} \sin{q_{0}} - 9.81m_{1} l_{1} (\sin{q_{0}} \cos{q_{1}} + \sin{q_{1}} \cos{q_{0}})) \\ -9.81 m_{1} l_{1} (\sin{q_{0}} \cos{q_{1}} + \sin{q_{1}} \cos{q_{0}}) \end{bmatrix}$

$B = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}$

$\tau_{ext} = \begin{bmatrix} 0 \\ 0 \end{bmatrix}$

In [16]:
class Controller(VectorSystem):
    """Defines a feedback controller for the double pendulum.

    The controller applies torques at the joints in order to:

    1) cancel out the dynamics of the double pendulum,
    2) make the first joint swing with the dynamics of a single pendulum, and
    3) drive the second joint towards zero.

    The magnitude of gravity for the imposed single pendulum dynamics is taken
    as a constructor argument.  So you can do fun things like pretending that
    gravity is zero, or even inverting gravity!
    """

    def __init__(self, multibody_plant, gravity):
        # 4 inputs (double pend state), 2 torque outputs.
        VectorSystem.__init__(self, 4, 2)
        self.plant = multibody_plant
        self.g = gravity

    def DoCalcVectorOutput(self, context, double_pend_state, unused, torque):
        # Extract manipulator dynamics.
        q = double_pend_state[:2]
        v = double_pend_state[-2:]
        (M, Cv, tauG, B, tauExt) = ManipulatorDynamics(self.plant, q, v)

        # Desired pendulum parameters.
        length = 1.0
        b = 0.1

        # Control gains for stabilizing the second joint.
        kp = 1
        kd = 0.1

        # Cancel double pend dynamics and inject single pend dynamics.
        torque[:] = (
            Cv
            - tauG
            - tauExt
            + M.dot(
                [
                    self.g / length * np.sin(q[0]) - b * v[0],
                    -kp * q[1] - kd * v[1],
                ]
            )
        )


def simulate(gravity=-9.8):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

    # Load the double pendulum from Universal Robot Description Format
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl(
        "package://underactuated/models/double_pendulum.urdf"
    )
    plant.Finalize()

    controller = builder.AddSystem(Controller(plant, gravity))
    builder.Connect(
        plant.get_state_output_port(), controller.get_input_port(0)
    )
    builder.Connect(
        controller.get_output_port(0), plant.get_actuation_input_port()
    )

    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    meshcat.Set2dRenderMode(xmin=-2.8, xmax=2.8, ymin=-2.8, ymax=2.8)

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    if running_as_notebook:
        simulator.set_target_realtime_rate(1.0)

    # Set the initial conditions
    context = simulator.get_mutable_context()
    context.SetContinuousState((0.2, 0.0, 0.2, 0.0))  # (θ₁, θ₂, θ̇₁, θ̇₂)

    # Simulate
    simulator.AdvanceTo(3.0)
    

In [17]:
simulate(-9.81)